# The effect of logging

QCoDeS comes with three levels of logging. 

- Logging can be completely off. This is the default.
- Logging can be on and log messages are saved to a file on disk.
- Logging can be on and log messages are saved to a file AND sent to Azure for telemetry.

This notebook performs a 2D measurement loop and times it for the three different cases in a worst-case-scenario situation where we log a message at every single step of the measurement.

In [1]:
import logging

import numpy as np

import qcodes as qc
from qcodes.logger import start_all_logging
from qcodes.dataset.measurements import Measurement
from qcodes.tests.instrument_mocks import DummyInstrument
from qcodes.dataset.plotting import plot_dataset
from qcodes.logger import flush_telemetry_traces
from qcodes.dataset.sqlite.database import initialise_or_create_database_at
from qcodes.dataset.experiment_container import new_experiment

## Set up the virtual experiment

We set up everything in a separate DB file, so you don't pollute your _real_ data with this benchmarking.

In [2]:
initialise_or_create_database_at('logging_benchmarking.db')
new_experiment('logging_benchmarking', 'the sample is in your mind')

Upgrading database: : 0it [00:00, ?it/s]
Upgrading database: : 0it [00:00, ?it/s]
Upgrading database, version 5 -> 6: : 0it [00:00, ?it/s]


logging_benchmarking#the sample is in your mind#1@C:\Users\wihpniel\src\qcodes\docs\examples\benchmarking\logging_benchmarking.db
---------------------------------------------------------------------------------------------------------------------------------

In [3]:
dac = DummyInstrument('dac', gates=['v1', 'v2'])
meter = DummyInstrument('meter', gates=['conductance'])
meter.conductance.unit = 'hbar/e^2'

In [4]:
def conductance_simulator():
    max_val = 1+np.pi/2+0.1
    return (np.sin(dac.v1()) + 
            np.arctan(dac.v2()) + 
            0.1*np.random.randn())/max_val

meter.conductance.get = conductance_simulator

In [5]:
meas = (Measurement()
        .register_parameter(dac.v1)
        .register_parameter(dac.v2)
        .register_parameter(meter.conductance, setpoints=[dac.v1, dac.v2]))


In [6]:
v1s = np.linspace(0, 5, 20)
v2s = np.linspace(-0.2, 0.2, 20)

## Version 1: no logging

What we'll eventually log is a message about whether the conductance is positive or negative. To make things fair, we still perform the evaluation of that condition in this version.

In [7]:
%%timeit

with meas.run() as datasaver:
    for v1 in v1s:
        dac.v1(v1)
        for v2 in v2s:
            dac.v2(v2)
            cond = meter.conductance()
            cond_pos = cond > 0
            datasaver.add_result((dac.v1, v1), 
                                 (dac.v2, v2),
                                 (meter.conductance, cond))

Starting experimental run with id: 1
Starting experimental run with id: 2
Starting experimental run with id: 3
Starting experimental run with id: 4
Starting experimental run with id: 5
Starting experimental run with id: 6
Starting experimental run with id: 7
Starting experimental run with id: 8
183 ms ± 13.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Version 2: Log to file only

In [8]:
qc.config.telemetry.enabled = False
start_all_logging()
log = logging.getLogger()

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : C:\Users\wihpniel\.qcodes\logs\command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active


In [9]:
%%timeit

with meas.run() as datasaver:
    for v1 in v1s:
        dac.v1(v1)
        for v2 in v2s:
            dac.v2(v2)
            cond = meter.conductance()
            cond_pos = cond > 0
            if cond_pos:
                log.info('Positive conductance')
            else:
                log.info('Negative conductance')
            datasaver.add_result((dac.v1, v1), 
                                 (dac.v2, v2),
                                 (meter.conductance, cond))

Starting experimental run with id: 9
Starting experimental run with id: 10
Starting experimental run with id: 11
Starting experimental run with id: 12
Starting experimental run with id: 13
Starting experimental run with id: 14
Starting experimental run with id: 15
Starting experimental run with id: 16
212 ms ± 9.04 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Version 3: Log to file and telemetric server

In [10]:
logging.shutdown()
qc.config.telemetry.enabled = True
start_all_logging()
log = logging.getLogger()

Activating auto-logging. Current session state plus future input saved.
Filename       : C:\Users\wihpniel\.qcodes\logs\command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active


In [13]:
%%timeit

with meas.run() as datasaver:
    for v1 in v1s:
        dac.v1(v1)
        for v2 in v2s:
            dac.v2(v2)
            cond = meter.conductance()
            cond_pos = cond > 0
            if cond_pos:
                log.info('Positive conductance')
            else:
                log.info('Negative conductance')
            datasaver.add_result((dac.v1, v1), 
                                 (dac.v2, v2),
                                 (meter.conductance, cond))

Starting experimental run with id: 33
Starting experimental run with id: 34
Starting experimental run with id: 35
Starting experimental run with id: 36
Starting experimental run with id: 37
Starting experimental run with id: 38
Starting experimental run with id: 39
Starting experimental run with id: 40
288 ms ± 22.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
